## Module 1 Homework
In this homework, we're going to download finance data from various sources and make simple calculations/analysis.



## Question 1: [Macro] Average growth of GDP in 2023
### What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [10]:
import pandas as pd
import pandas_datareader as pd 

import time
from datetime import date

In [12]:
end = date.today()
start = date(year=end.year-70, month=end.month, day=end.day)

In [15]:
df = pdr.DataReader('GDPC1', 'fred', start=start, end=end)
df.head()

,GDPC1
DATE,
1954-07-01,2880.482
1954-10-01,2936.852
1955-01-01,3020.746
1955-04-01,3069.910
1955-07-01,3111.379


In [16]:
# YoY growth
df['YoY'] = df['GDPC1'] / df['GDPC1'].shift(4) - 1
df.head()

,GDPC1,YoY
DATE,,
1954-07-01,2880.482,NaN
1954-10-01,2936.852,NaN
1955-01-01,3020.746,NaN
1955-04-01,3069.910,NaN
1955-07-01,3111.379,0.080159


In [19]:
# filtered only 2023 year
df_2023 = df.loc['2023-01-01':'2023-12-31']
df_2023.head()

,GDPC1,YoY
DATE,,
2023-01-01,22112.329,0.017179
2023-04-01,22225.350,0.023825
2023-07-01,22490.692,0.029269
2023-10-01,22679.255,0.031345


In [21]:
average_yoy_2023 = df_2023['YoY'].mean()

# round
average_yoy_2023_rounded = round(average_yoy_2023 * 100, 1)

print("Average growth of GDP in 2023: {}%".format(average_yoy_2023_rounded))

Average growth of GDP in 2023: 2.5%


### Answer:
- 2.5

## Question 2. [Macro] Inverse "Treasury Yield"
### Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [28]:
end = date.today()
start = '2000-01-01'

In [29]:
start

'2000-01-01'

In [39]:
df_dgs2 = pdr.DataReader('DGS2', 'fred', start=start, end=end)

df_dgs2

,DGS2
DATE,
2000-01-03,6.38
2000-01-04,6.30
2000-01-05,6.38
2000-01-06,6.35
2000-01-07,6.31
...,...
2024-04-17,4.93
2024-04-18,4.98
2024-04-19,4.97


In [33]:
df_dgs10 = pdr.DataReader('DGS10', 'fred', start=start, end=end)

df_dgs10

,DGS10
DATE,
2000-01-03,6.58
2000-01-04,6.49
2000-01-05,6.62
2000-01-06,6.57
2000-01-07,6.52
...,...
2024-04-16,4.67
2024-04-17,4.59
2024-04-18,4.64


In [40]:
df_dgs = df_dgs2.join(df_dgs10, on = 'DATE', how='inner')

df_dgs

,DGS2,DGS10
DATE,,
2000-01-03,6.38,6.58
2000-01-04,6.30,6.49
2000-01-05,6.38,6.62
2000-01-06,6.35,6.57
2000-01-07,6.31,6.52
...,...,...
2024-04-16,4.97,4.67
2024-04-17,4.93,4.59
2024-04-18,4.98,4.64


In [42]:
df_dgs['dgs10-dgs2'] = df_dgs['DGS10'] - df_dgs['DGS2']

df_dgs

,DGS2,DGS10,dgs10-dgs2
DATE,,,
2000-01-03,6.38,6.58,0.20
2000-01-04,6.30,6.49,0.19
2000-01-05,6.38,6.62,0.24
2000-01-06,6.35,6.57,0.22
2000-01-07,6.31,6.52,0.21
...,...,...,...
2024-04-16,4.97,4.67,-0.30
2024-04-17,4.93,4.59,-0.34
2024-04-18,4.98,4.64,-0.34


In [44]:
df_dgs['dgs10-dgs2'].min().round(1)

-1.1

### Answer:
- -1.1

## Question 3. [Index] Which Index is better recently?
### Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [76]:
import yfinance as yf

In [77]:
start = '2019-04-09'
end = '2024-04-09'

In [78]:
sp500 = yf.download('^GSPC', start=start, end=end)

sp500_growth = (sp500['Adj Close'][-1] / sp500['Adj Close'][0] - 1) * 100

[*********************100%***********************]  1 of 1 completed


In [79]:
sp500_growth

80.75151917783086

In [80]:
ipc_mexico = yf.download('^MXX', start=start, end=end)

ipc_mexico_growth = (ipc_mexico['Adj Close'][-1] / ipc_mexico['Adj Close'][0] - 1) * 100

[*********************100%***********************]  1 of 1 completed


In [81]:
ipc_mexico_growth

28.43377484045295

### Answer:
- 81%

## Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks
### Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [82]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

result = {}

for ticker in tickers:
    data = yf.download(ticker, start='2023-01-01', end='2023-12-31')
    
    max_adj_close = data['Adj Close'].max()
    min_adj_close = data['Adj Close'].min()
    
    #  (max - min) / max
    range_ratio = (max_adj_close - min_adj_close) / max_adj_close
    
    # round 2
    result[ticker] = round(range_ratio, 2)

# Найти наибольший диапазон отношения
largest_range_ratio = max(result.values())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [84]:
result

{'2222.SR': 0.21,
 'BRK-B': 0.21,
 'AAPL': 0.37,
 'MSFT': 0.42,
 'GOOG': 0.39,
 'JPM': 0.28}

In [83]:
largest_range_ratio

0.42

### Answer: 
- 0.42

## Question 5. [Stocks] Dividend Yield
### Find the largest dividend yield for the same set of stocks

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [114]:
div_data = yf.Ticker('JPM')
div_data.dividends.loc['2023-01-01':'2023-12-31']

Date
2023-01-05 00:00:00-05:00    1.00
2023-04-05 00:00:00-04:00    1.00
2023-07-05 00:00:00-04:00    1.00
2023-10-05 00:00:00-04:00    1.05
Name: Dividends, dtype: float64

In [115]:
div_data = yf.Ticker('2222.SR')
div_data.dividends.loc['2023-01-01':'2023-12-31']

Date
2023-03-15 00:00:00+03:00    0.302364
2023-05-17 00:00:00+03:00    0.302400
2023-09-11 00:00:00+03:00    0.153000
2023-11-15 00:00:00+03:00    0.153000
Name: Dividends, dtype: float64

In [116]:
div_data = yf.Ticker('BRK-B')
div_data.dividends.loc['2023-01-01':'2023-12-31']

Series([], Name: Dividends, dtype: float64)

In [117]:
div_data = yf.Ticker('AAPL')
div_data.dividends.loc['2023-01-01':'2023-12-31']

Date
2023-02-10 00:00:00-05:00    0.23
2023-05-12 00:00:00-04:00    0.24
2023-08-11 00:00:00-04:00    0.24
2023-11-10 00:00:00-05:00    0.24
Name: Dividends, dtype: float64

In [ ]:
div_data = yf.Ticker('2222.SR')
div_data.dividends.loc['2023-01-01':'2023-12-31']

In [124]:
import yfinance as yf

# list of tickets
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

dividend_yield = {}

# loading dividend data for each tickets and count agg close price in 2023
for ticker in tickers:
    try:
        # loading dividend data for particular tickets in 2023 
        div_data = yf.Ticker(ticker).dividends
        div_2023 = div_data.loc['2023']

        # Check if there's div data for particular tickets
        if not div_2023.empty:
            # Loading close price data in 2023 
            close_price_data = yf.Ticker(ticker).history(start='2023-01-01', end='2023-12-31')['Close']

            # Check if there's close price data for particular tickets
            if not close_price_data.empty:
                # Calculate agg close price in 2023
                average_close_price_2023 = close_price_data.mean()

                # Summarize of ticket's dividend in 2023
                total_dividends = div_2023.sum()

                # Dividend ratio for particular ticket
                yield_percentage = (total_dividends / average_close_price_2023) * 100

                dividend_yield[ticker] = yield_percentage
    except Exception as e:
        print(f"Error while reading ticket data {ticker}: {e}")

if dividend_yield:
    # Find max dividend yeald
    max_yield = max(dividend_yield.values())

    max_yield_rounded = round(max_yield, 1)

    print("Max div yield in 2023: {}%".format(max_yield_rounded))
else:
    print("Error dividend data")


Max div yield in 2023: 2.9%


## Question 6. [Exploratory] Investigate new metrics
### Free text answer

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

## Question 7. [Exploratory] Time-driven strategy description around earnings releases
### Free text answer

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.